## Importuri necesare 

In [29]:
import os
import numpy as np
import matplotlib.pyplot as plt

## Citirea datelor

In [30]:
import pandas as pd
data_path = '../Lab 5/'
train_data_df = pd.read_csv(os.path.join(data_path, 'train_data.csv'))

In [31]:
for col in train_data_df.columns:
    print(col)

language
text
label


In [32]:
train_data_df

,language,text,label
0,dansk,"\nDette er et fremragende initiativ, og jeg st...",Ireland
1,dansk,"\nHr. formand, jeg er sikker på, at alle her e...",Ireland
2,dansk,"\nHr. formand, folk på den nordlige halvkugle ...",England
3,dansk,"\nHr. formand, med forbehold af nogle få ændri...",England
4,dansk,"\n\n - Hr. formand, jeg må protestere mod de...",England
...,...,...,...
41565,Nederlands,"\nMijnheer de Voorzitter, juridisch gezien is ...",England
41566,Nederlands,"\n\n . Mijnheer de Voorzitter, het is niet ...",Scotland
41567,Nederlands,"\nAls afgevaardigde van Ierland, het 'voedsele...",Ireland
41568,Nederlands,"\nMijnheer de Voorzitter, het is niet onterech...",England


In [33]:
print('Distributia etichetelor in datele de antrenare \n', train_data_df['label'].value_counts())

Distributia etichetelor in datele de antrenare 
 England     22700
Ireland     10535
Scotland     8335
Name: label, dtype: int64


In [34]:
# fiecare limba este reprezentata in mod egal
train_data_df['language'].value_counts()

dansk         8314
Deutsch       8314
español       8314
italiano      8314
Nederlands    8314
Name: language, dtype: int64

## Codificarea etichetelor din string in int

In [35]:
#Exercitiu: codificam etichetele / labels in valori cu numere intregi de la 0 la N
# codificam etichetele / labels in valori cu numere intregi dela 0 la N
# ar putea fi exercitiu la clasa ca sa se obisnuiasca cu dictionare
etichete_unice = train_data_df['label'].unique()
print(etichete_unice)
label2id = {}
id2label = {}
for idx, eticheta in enumerate(etichete_unice):
    label2id[eticheta] = idx
    id2label[idx] = eticheta

print(label2id)
print(id2label)

['Ireland' 'England' 'Scotland']
{'Ireland': 0, 'England': 1, 'Scotland': 2}
{0: 'Ireland', 1: 'England', 2: 'Scotland'}


In [36]:
#Exercitiu: aplicati dictionarul label2id peste toate etichetele din train
labels = []

for i in train_data_df['label']:
    labels.append(label2id[i])

print(labels[:10])

[0, 0, 1, 1, 1, 0, 1, 1, 1, 2]


## Pre-procesarea datelor

- extragem informațiile necesare din text
- eliminăm semnele de punctuație
- facem tokenizare (împărțire în cuvinte)
- vezi tokenizatori in [NLTK](https://www.nltk.org/api/nltk.tokenize.html)
- vezi tokenizatori in [spaCy](https://spacy.io/)
- vezi tokenizatori precum [BPE, WordPiece, SentencePiece](https://huggingface.co/docs/transformers/tokenizer_summary)

In [37]:
import re

def proceseaza(text):
    """Functie simpla de procesare a textului.
    Sugestii:
    - cum procesati \n new lines? (vezi functia strip())
    - cum procesati empty token ''
    - puteti introduce un tokenizator din nltk
        https://www.nltk.org/api/nltk.tokenize.html
    - puteti elimina sau pastra doar stop-words
    """
    text = re.sub("[-.,;:!?\"\'\/()_*=`]", "", text)
    text_in_cuvinte = text.strip().split(' ')
    return text_in_cuvinte

# cuvintele rezultate din functia de preprocesare:
exemple_italian = train_data_df[train_data_df['language'] == 'italiano']
print(exemple_italian)


       language                                               text     label
24942  italiano  \nSi tratta di un'eccellente iniziativa e cond...   Ireland
24943  italiano  \nSignor Presidente, tutti i presenti sanno ce...   Ireland
24944  italiano  \nSignor Presidente, nel Nord del mondo si pen...   England
24945  italiano  \nSignor Presidente, fatti salvi alcuni emenda...   England
24946  italiano  \n\n   – Signor Presidente, desidero protestar...   England
...         ...                                                ...       ...
33251  italiano  \nSignor Presidente, l' obiettivo giuridico de...   England
33252  italiano  \n\n    Signor Presidente, non spetta a me tog...  Scotland
33253  italiano  \nVenendo dall'Irlanda, l'"isola del cibo", so...   Ireland
33254  italiano  \nSignor Presidente, è opportuno che io interv...   England
33255  italiano  \n\n   – E’ d’accordo il Presidente in carica ...   Ireland

[8314 rows x 3 columns]


In [38]:
text_italian = exemple_italian['text'].iloc[0]
print(proceseaza(text_italian)[:13])

['Si', 'tratta', 'di', 'uneccellente', 'iniziativa', 'e', 'condivido', 'totalmente', 'lobiettivo', 'di', 'mettere', 'in', 'rilievo']


### Aplicam functia de preprocesarea intregului set de date

In [39]:
# Exercitiu: aplicati functia pe fiecare exemplu de antrenare
data = []

for text in train_data_df['text']:
    data.append(proceseaza(text))

print(data[0])

['Dette', 'er', 'et', 'fremragende', 'initiativ', 'og', 'jeg', 'støtter', 'fuldt', 'ud', 'målet', 'om', 'at', 'fremhæve', 'idrættens', 'pædagogiske', 'rolle', 'Der', 'kan', 'opnås', 'store', 'fordele', 'ved', 'at', 'etablere', 'partnerskaber', 'mellem', 'undervisningssektorens', 'og', 'idrættens', 'organisationer\n\nSom', 'irsk', 'medlem', 'er', 'jeg', 'specielt', 'glad', 'for', 'at', 'forslaget', 'går', 'ud', 'på', 'at', 'dedikere', '2004', 'til', 'Idrættens', 'Pædagogiske', 'Dimension', 'Det', 'falder', 'sammen', 'med', 'det', 'irske', 'formandskab', 'i', 'første', 'halvdel', 'af', '2004', 'og', 'jeg', 'kan', 'forsikre', 'mine', 'kolleger', 'om', 'at', 'Irland', 'vil', 'spille', 'en', 'fremtrædende', 'rolle', 'i', 'mange', 'af', 'de', 'planlagte', 'initiativer', 'Irland', 'har', 'en', 'lang', 'tradition', 'for', 'sport', 'særligt', 'på', 'amatør', 'og', 'frivillig', 'basis\n\nVi', 'er', 'alle', 'klar', 'over', 'at', 'en', 'række', 'store', 'sportsbegivenheder', 'løber', 'af', 'stabel

## Împărțirea datelor în train, validare și test

O împărțire brutală poate fi cea în funcție de ordinea în care apar datele.
- cum poate afecat acest tip de împărțire rezultatele dacă datele sunt sortate după etichetă?

In [40]:
print(len(data))

# putem imparti datele de antrenare astfel:
# 20% date de test din total 
# 15% date de validare din ce ramane dupa ce scoatem datele de test 

nr_test = int(20/100 * len(train_data_df))
print("Nr de date de test: ", nr_test)

nr_ramase = len(data) - nr_test
nr_valid = int(15/100 * nr_ramase)
print("Nr de date de validare: ", nr_valid)

nr_train = nr_ramase - nr_valid
print("Nr de date de antrenare: ", nr_train)



41570
Nr de date de test:  8314
Nr de date de validare:  4988
Nr de date de antrenare:  28268


In [41]:
# facem impartirea in ordinea in care apar datele

train_data = data[:nr_train]
train_labels = labels[:nr_train]

valid_data = data[nr_train : nr_train + nr_valid]
valid_labels = labels[nr_train : nr_train + nr_valid]

test_data = data[nr_train + nr_valid: ]
test_labels = labels[nr_train + nr_valid:]


print(f'Nr de exemple de train {len(train_labels)}')
print(f'Nr de exemple de validare {len(valid_labels)}')
print(f'Nr de exemple de test {len(test_labels)}')

Nr de exemple de train 28268
Nr de exemple de validare 4988
Nr de exemple de test 8314


In [42]:
#Exercitiu: vedeti cum sunt distribuite etichetele in train, valid, test (incercati bincount)

# sklearn train_test_split

## Bag of Words

În cadrul acestei secțiuni vom face numărarea aparițiilor tuturor cuvintelor din datele noastre. Pentru o evaluare justă nu ar fi indicat să includem si cuvintele din datele de test.

In [43]:
from collections import Counter

# vom folosi o structura de date nativa din python
# care functioneaza ca un dictionar care numara elementele hashable
# dintr-o colectie
ctr = Counter(['eggs', 'ham', 'eggs', 'egg'])
print(ctr)
print(ctr['bacon'])
print(ctr['eggs'])

ctr = Counter([(0,1), (0,0), (0,1)])
print(ctr[(0,0)])
print(ctr[(2,2)])


Counter({'eggs': 2, 'ham': 1, 'egg': 1})
0
2
1
0


In [44]:
ctr = Counter(data[9234])
print(ctr.keys())
print(ctr.values())
# care este cel mai frecvent cuvant in data[5]?

dict_keys(['Ich', 'habe', 'gegen', 'diesen', 'Bericht', 'gestimmt', 'um', 'dem', 'Ausschuss', 'für', 'Haushaltskontrolle', 'mehr', 'Zeit', 'zu', 'geben', 'untersuchen', 'ob', 'Parlament', 'seine', 'Gebäude', 'in', 'Straßburg', 'viel', 'berechnet', 'wurde', 'Es', 'ist', 'unbedingt', 'erforderlich', 'dass', 'diese', 'höchst', 'unbefriedigende', 'Angelegenheit', 'umfassend', 'untersucht', 'wird', 'Das', 'muss', 'alles', 'seiner', 'Macht', 'Stehende', 'tun', 'das', 'Geld', 'der', 'Steuerzahler', 'schützen'])
dict_values([1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 4, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


### Frecventa cuvintelor din setul de antrenare

In [45]:
counter = Counter()

for cuvinte in train_data:
    counter.update(cuvinte)

print(counter.most_common(10))


[('de', 182565), ('la', 115758), ('der', 96800), ('que', 96475), ('en', 94038), ('die', 79963), ('at', 78111), ('a', 64202), ('i', 63742), ('og', 60776)]


### Reprezentarea datelor sub forma vectoriala

- sa presupunem ca folosim primele N cuvinte non-nule ca caracteristici pentru fiecare text


In [46]:
N = 10
cuvinte_caracteristice = []
for cuvant, frecventa in counter.most_common(N):
    if cuvant.strip():
        cuvinte_caracteristice.append(cuvant)
print(cuvinte_caracteristice)

['de', 'la', 'der', 'que', 'en', 'die', 'at', 'a', 'i', 'og']


- fiecarui cuvant îi atribuim un id în funcție de poziția pe care se află
- ordinea in care sunt stocate cheile intr-un dictionar este arbitrara 
- iar o lista este un obiect mutabil in care ordinea elementelor se poate schimba oricand
- cel mai sigur este sa construim o mapare intre cuvinte si un id care sa reprezinte pozitia in vectorul de caracteristici


In [47]:
word2id = {}
id2word = {}
for idx, cuv in enumerate(cuvinte_caracteristice):
    word2id[cuv] = idx
    id2word[idx] = cuv

print(word2id)
print(id2word)

{'de': 0, 'la': 1, 'der': 2, 'que': 3, 'en': 4, 'die': 5, 'at': 6, 'a': 7, 'i': 8, 'og': 9}
{0: 'de', 1: 'la', 2: 'der', 3: 'que', 4: 'en', 5: 'die', 6: 'at', 7: 'a', 8: 'i', 9: 'og'}


- cand trebuie sa reprezentam un text sub forma vectoriala, ne raportam doar la cuvintele cheie pe care le folosim ca caracteristici
- id-ul reprezinta pozitia in vector unde vom stoca aparitiile fiecarui cuvant

In [48]:
# 1. numaram toate cuvintele din text
ctr = Counter(train_data[1])

# 2. prealocam un array care va reprezenta caracteristicel noastre
features = np.zeros(len(cuvinte_caracteristice))

# 3. umplem array-ul cu valorile obtinute din counter
# fiecare pozitie din array trebuie sa reprezinte frecventa
# aceluiasi cuvant in toate textele
for idx in range(0, len(features)):
    # obtinem cuvantul pentru pozitia idx
    cuvant = id2word[idx]
    # asignam valoarea corespunzatoare frecventei cuvantului
    features[idx] = ctr[cuvant]

print(features)
print([id2word[idx] for idx in range(0, len(features))])

[4. 0. 5. 0. 9. 0. 4. 0. 3. 9.]
['de', 'la', 'der', 'que', 'en', 'die', 'at', 'a', 'i', 'og']


### Punem totul cap la cap sub forma de functii

In [49]:
def count_most_common(how_many, texte_preprocesate):
    """Functie care returneaza cele mai frecvente cuvinte.
    """
    
    for cuvinte in texte_preprocesate:
        counter.update(cuvinte)
        
    cuvinte_caracteristice = []
    for cuvant, frecventa in counter.most_common(how_many):
        if cuvant.strip():
            cuvinte_caracteristice.append(cuvant)

    return cuvinte_caracteristice

In [50]:
def build_id_word_dicts(cuvinte_caracteristice):
    '''Dictionarele word2id si id2word garanteaza o ordine
    pentru cuvintele caracteristice.
    '''
    word2id = {}
    id2word = {}
    for idx, cuv in enumerate(cuvinte_caracteristice):
        word2id[cuv] = idx
        id2word[idx] = cuv

    return word2id, id2word

In [51]:
def featurize(text_preprocesat, id2word):
    """Pentru un text preprocesat dat si un dictionar
    care mapeaza pentru fiecare pozitie ce cuvant corespunde,
    returneaza un vector care reprezinta
    frecventele fiecarui cuvant.
    """
    ctr = Counter(text_preprocesat)
    features = np.zeros(len(id2word))
    
    for idx in range(0, len(features)):
        # obtinem cuvantul pentru pozitia idx
        cuvant = id2word[idx]
        # asignam valoarea corespunzatoare frecventei cuvantului
        features[idx] = ctr[cuvant]

    return features

In [52]:
def featurize_multi(texte, id2word):
    '''Pentru un set de texte preprocesate si un dictionar
    care mapeaza pentru fiecare pozitie ce cuvant corespunde,
    returneaza matricea trasaturilor tuturor textelor.
    '''
    all_features = []
    for text in texte:
        all_features.append(featurize(text, id2word))
    return np.array(all_features)

In [53]:
cuvinte_caracteristice = count_most_common(20, train_data)
print(cuvinte_caracteristice)

['de', 'la', 'der', 'que', 'en', 'die', 'at', 'a', 'i', 'og', 'und', 'y', 'el', 'in', 'er', 'den', 'los', 'for', 'af', 'til']


In [58]:
cuvinte_caracteristice = count_most_common(100, train_data)
print(len(cuvinte_caracteristice))
print(cuvinte_caracteristice)
word2id, id2word = build_id_word_dicts(cuvinte_caracteristice)

X_train = featurize_multi(train_data, id2word)
X_valid = featurize_multi(valid_data, id2word)
X_test = featurize_multi(test_data, id2word)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)


99
['de', 'la', 'der', 'que', 'en', 'die', 'at', 'a', 'i', 'og', 'und', 'y', 'el', 'in', 'er', 'den', 'los', 'for', 'af', 'til', 'del', 'di', 'det', 'un', 'es', 'las', 'zu', 'se', 'una', 'på', 'om', 'con', 'har', 'che', 'e', 'von', 'som', 'ikke', 'vi', 'für', 'med', 'para', 'no', 'ist', 'dass', 'por', 'wir', 'nicht', 'al', 'das', 'vil', 'auf', 'ha', 'jeg', 'et', 'il', 'eine', 'des', 'ich', 'dem', 'werden', 'mit', 'per', 'im', 'Det', 'lo', 'su', 'sich', 'kan', 'ein', 'Jeg', 'daß', 'wird', 'Ich', 'si', 'alle', 'sobre', 'Presidente', 'más', 'este', 'skal', 'è', 'como', 'sie', 'haben', 'le', 'auch', 'sind', 'Comisión', 'fra', 'han', 'Europa', 'non', 'um', 'esta', 'als', 'della', 'Parlamento', 'wie']
(28268, 99)
(4988, 99)
(8314, 99)


In [59]:
from sklearn.metrics import accuracy_score
from sklearn import svm

model = svm.LinearSVC(C=1)

model.fit(X_train, train_labels)
vpreds = model.predict(X_valid)
tpreds = model.predict(X_test)

print(accuracy_score(valid_labels, vpreds))
print(accuracy_score(test_labels, tpreds))

0.5467121090617482
0.5093817656964157


/usr/local/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Exemplu de lucru cu stop words in nltk

In [56]:
# luat in calcul cuvintele functionale, pot creste sau scadea acuratetea
# in cazul de fata, cuvintele functionale (stop words) indica elemente de gramatica
# care sunt specifice textelor de la Scotieni, Englezi, Irlandezi
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
print(stopwords.words('french'))

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

[nltk_data] Downloading package stopwords to /Users/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
